In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('shark_clean.csv')

In [3]:
df.head()

,Unnamed: 0,Date,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Time,Species
0,0,09-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,Fatal,Y,10h45,Not Available
1,1,07-Oct-2020,2020.0,Unprovoked,USA,Florida,"Miami Beach, Miami-Dade County",Body Surfing,M,Survived,N,Afternoon,Blacktip shark
2,2,04-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,"Toms Surf break, Hamersley Pool, North Beach, ...",Surfing,M,No Injury,N,10h45,Bronze whaler 1.5m
3,4,20-Sep-2020,2020.0,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,M,Survived,N,10h30,Bull shark
4,5,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,M,Survived,N,16h00,"""whitetip shark"""


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5889 entries, 0 to 5888
Data columns (total 12 columns):
Date           5889 non-null object
Year           5889 non-null float64
Type           5889 non-null object
Country        5889 non-null object
Area           5889 non-null object
Location       5889 non-null object
Activity       5889 non-null object
Sex            5889 non-null object
Injury         5889 non-null object
Fatal (Y/N)    5889 non-null object
Time           5889 non-null object
Species        5889 non-null object
dtypes: float64(1), object(11)
memory usage: 552.2+ KB


In [6]:
for i in df:
    print(i, '/n')
    
    print(df[i].value_counts(normalize=True).head())
    print('-----------------')

Date /n
1957    0.001528
1942    0.001189
1950    0.001189
1941    0.001189
1958    0.001189
Name: Date, dtype: float64
-----------------
Year /n
2015.0    0.022075
2017.0    0.021056
2016.0    0.020207
0.0       0.020207
2011.0    0.020037
Name: Year, dtype: float64
-----------------
Type /n
Unprovoked      0.797928
Provoked        0.099677
Watercraft      0.056546
Sea Disaster    0.038377
Invalid         0.004245
Name: Type, dtype: float64
-----------------
Country /n
USA                 0.365597
AUSTRALIA           0.213619
SOUTH AFRICA        0.088130
PAPUA NEW GUINEA    0.022075
NEW ZEALAND         0.021056
Name: Country, dtype: float64
-----------------
Area /n
Florida            0.173374
New South Wales    0.076414
Not Available      0.063678
Queensland         0.051622
Hawaii             0.047207
Name: Area, dtype: float64
-----------------
Location /n
Not Available                       0.075565
New Smyrna Beach, Volusia County    0.030056
Daytona Beach, Volusia County       0

In [9]:
df['Date'].value_counts()

1957                                                                9
1942                                                                7
1950                                                                7
1941                                                                7
1958                                                                7
No date                                                             6
1949                                                                6
1956                                                                6
1970s                                                               5
Aug-1956                                                            5
1954                                                                5
05-Oct-2003                                                         5
1959                                                                5
28-Jul-1995                                                         5
12-Apr-2001         

In [18]:
df['Year'] = [i[-4:] for i in df['Date']]

In [21]:
df['Month'] = [i[3:6] for i in df['Date']]

In [22]:
df.head()

,Date,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Time,Species,Month
0,09-Oct-2020,2020,Unprovoked,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,Fatal,Y,10h45,Not Available,Oct
1,07-Oct-2020,2020,Unprovoked,USA,Florida,"Miami Beach, Miami-Dade County",Body Surfing,M,Survived,N,Afternoon,Blacktip shark,Oct
2,04-Oct-2020,2020,Unprovoked,AUSTRALIA,Western Australia,"Toms Surf break, Hamersley Pool, North Beach, ...",Surfing,M,No Injury,N,10h45,Bronze whaler 1.5m,Oct
3,20-Sep-2020,2020,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,M,Survived,N,10h30,Bull shark,Sep
4,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,M,Survived,N,16h00,"""whitetip shark""",Sep
